In [1]:
import sys
import os
import json
import numpy as np
sys.path.append('TradeEngine')
from TradeEngine.option.hProcessOption import hProcess_option_Deribit, hProcess_option_Bybit, hProcess_option_Okx
from TradeEngine.option.hAggregatorOption import hAggregator_Option


ModuleNotFoundError: No module named 'TradeEngine'

### Content
- Deribit hProcess
- OKX hProcess
- Bybit hProcess
- Aggregator


### Forking

- The most import thing to modify is #option_data# dictionary. The rest may be left unchanged (suppouse you have a dictionary with all options combined)


# Deribit example

In [2]:
indexPrice = 41674   

expiration_ranges = np.array([1.0, 7.0, 35.0])
price_levels = np.array([0, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 100.0])
options_data_raw= json.load(open("data\option_deribit.json"))

deribit = hProcess_option_Deribit(expiration_ranges, price_levels)
deribit.input_oi(raw_data=options_data_raw, indexPrice=indexPrice)

print( "The list of dataframes per expiration_countdown: ", list(deribit.df_call.keys()))

deribit.df_call['1']        # Each dataframe contains a single row, with will should be concatenated to a aggregators dataframe
deribit.df_put['1']


The list of dataframes per expiration_countdown:  ['1', '1_7', '35', '7_35']


,-25.0,-10.0,-5.0,-2.0,-1.0,-0.5,0.0,0.5,1.0,2.0,5.0,10.0,25.0,100.0
timestamp,,,,,,,,,,,,,,
2024-01-22,90.6,211.9,136.7,1668.6,63.3,0.0,5.0,5.0,7.7,9.7,50.6,131.8,68.1,0.0


# OKX Example

In [3]:
indexPrice = 41674

expiration_ranges = np.array([1.0, 7.0, 35.0])
price_levels = np.array([0, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 100.0])
options_data_raw= json.load(open("data\option_okx.json"))

okex = hProcess_option_Okx(expiration_ranges, price_levels)
okex.input_oi(raw_data=options_data_raw, indexPrice=indexPrice)

print( "The list of dataframes per expiration_countdown: ", list(deribit.df_call.keys()))

okex.df_call['1']        # Each dataframe contains a single row, with will should be concatenated to a aggregators dataframe
okex.df_put['1']

The list of dataframes per expiration_countdown:  ['1', '1_7', '35', '7_35']


,-25.0,-10.0,-5.0,-2.0,-1.0,-0.5,0.0,0.5,1.0,2.0,5.0,10.0,25.0,100.0
timestamp,,,,,,,,,,,,,,
2024-01-22,0.0,24.57,74.96,122.03,142.1,13.36,105.75,9.58,7.84,4.36,0.0,0.0,0.0,0.0


# Bybit Example

In [4]:
indexPrice = 41674

expiration_ranges = np.array([1.0, 7.0, 35.0])
price_levels = np.array([0, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 100.0])
options_data_raw= json.load(open("data\option_bybit.json"))

bybit = hProcess_option_Bybit(expiration_ranges, price_levels)
bybit.input_oi(raw_data=options_data_raw,  indexPrice=indexPrice)


print( "The list of dataframes per expiration_countdown: ", list(deribit.df_call.keys()))

bybit.df_call['1']        # Each dataframe contains a single row, with will should be concatenated to a aggregators dataframe
bybit.df_put['1_7']

The list of dataframes per expiration_countdown:  ['1', '1_7', '35', '7_35']


,-25.0,-10.0,-5.0,-2.0,-1.0,-0.5,0.0,0.5,1.0,2.0,5.0,10.0,25.0,100.0
timestamp,,,,,,,,,,,,,,
2024-01-22,258.97,268.7,296.38,233.74,197.2,0.0,2.77,7.89,158.43,6.92,150.83,308.28,78.78,2.1


# Aggregator

In [7]:
import time
start = time.time()


indexPrice = 41674
expiration_ranges = np.array([1.0, 7.0, 35.0])
price_levels : np.array([0, 0.5, 1.0, 2.0, 5.0, 10.0, 25.0, 100.0])

processes = [
    hProcess_option_Deribit(expiration_ranges, price_levels), 
    hProcess_option_Okx(expiration_ranges, price_levels), 
    hProcess_option_Bybit(expiration_ranges, price_levels)
             ]

raw_data = [
    json.load(open("data\option_deribit.json")),
    json.load(open("data\option_okx.json")),
    json.load(open("data\option_bybit.json")),
]

agg = hAggregator_Option(processes)
agg.input_data(raw_data, indexPrice)
time.sleep(1)
agg.input_data(raw_data, indexPrice)
print("Elapsed: ", start - time.time())

Elapsed:  -2.079468250274658


In [8]:
agg.snapshot


,call_1_-25.0,call_1_-10.0,call_1_-5.0,call_1_-2.0,call_1_-1.0,call_1_-0.5,call_1_0.0,call_1_0.5,call_1_1.0,call_1_2.0,...,put_7_35_-1.0,put_7_35_-0.5,put_7_35_0.0,put_7_35_0.5,put_7_35_1.0,put_7_35_2.0,put_7_35_5.0,put_7_35_10.0,put_7_35_25.0,put_7_35_100.0
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-22 00:15:31.706000896+00:00,31.3,2.25,976.08,86.5,89.31,0.1,105.35,69.91,537.59,117.01,...,266.1,0.0,27.06,0.0,839.49,17.7,401.01,1263.39,1436.43,1835.39
2024-01-22 00:15:33.218808576+00:00,31.3,2.25,976.08,86.5,89.31,0.1,105.35,69.91,537.59,117.01,...,266.1,0.0,27.06,0.0,839.49,17.7,401.01,1263.39,1436.43,1835.39
